In [14]:
import pandas as pd
import requests
import json

In [60]:
loc = r'.\stats\\'
season = '2021-22'

# Basic Defensive Statistics

Gets basic defense stats from nba.com

In [61]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://www.nba.com/'
}

# Retrieve both defensive dashboard and defensive impact
r_dd = requests.get('https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom=&DateTo=&DefenseCategory=Overall&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)
r_di = requests.get('https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&PlayerExperience=&PlayerOrTeam=Player&PlayerPosition=&PtMeasureType=Defense&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [62]:
r_dd_dict = json.loads(r_dd.content)
r_di_dict = json.loads(r_di.content)

dd_df = pd.DataFrame(data=r_dd_dict['resultSets'][0]['rowSet'], columns=r_dd_dict['resultSets'][0]['headers'])
di_df = pd.DataFrame(data=r_di_dict['resultSets'][0]['rowSet'], columns=r_di_dict['resultSets'][0]['headers'])

dd_df.sort_values(by='PLAYER_NAME', inplace=True)
di_df.sort_values(by='PLAYER_NAME', inplace=True)

In [63]:
# Remove players not in both dataframes
diff = list(set(di_df['PLAYER_ID'].tolist()) - set(dd_df['CLOSE_DEF_PERSON_ID'].tolist()))

di_df = di_df[~di_df['PLAYER_ID'].isin(diff)]

# Combine dataframes
def_data = dd_df.join(di_df[di_df.columns[5:11]])

# Drop unnecessary columns
def_data.drop(columns=['PLAYER_LAST_TEAM_ID', 'PLAYER_LAST_TEAM_ABBREVIATION', 'PLAYER_POSITION', 'FREQ'], inplace=True)
def_data.rename(columns={'CLOSE_DEF_PERSON_ID' : 'PLAYER_ID'}, inplace=True)

def_data.head()

,PLAYER_ID,PLAYER_NAME,AGE,GP,G,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS,W,L,MIN,STL,BLK,DREB
128,203932,Aaron Gordon,26.0,47,47,4.91,11.23,0.438,0.462,-0.024,28.0,16.0,22.9,1.6,0.5,3.3
555,1630565,Aaron Henry,22.0,5,5,0.40,1.60,0.250,0.405,-0.155,10.0,19.0,15.2,0.3,0.4,3.0
357,1628988,Aaron Holiday,25.0,36,36,3.00,6.31,0.476,0.435,0.041,24.0,24.0,33.2,0.7,0.9,7.7
472,1630174,Aaron Nesmith,22.0,36,36,2.06,3.94,0.521,0.430,0.091,21.0,21.0,19.5,0.6,0.0,1.8
211,1630598,Aaron Wiggins,23.0,31,31,4.68,9.32,0.502,0.457,0.045,15.0,32.0,33.5,0.7,0.2,4.8


# Hustle Statistics

Gets contested shots (2p and 3p), deflections, charges drawn, defensive boxouts, %boxout

In [64]:
r = requests.get('https://stats.nba.com/stats/leaguehustlestatsplayer?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [65]:
r_dict = json.loads(r.content)

df = pd.DataFrame(data=r_dict['resultSets'][0]['rowSet'], columns=r_dict['resultSets'][0]['headers'])
df.drop(columns=['TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'G', 'MIN',
                'SCREEN_AST_PTS', 'SCREEN_ASSISTS', 'OFF_LOOSE_BALLS_RECOVERED', 'DEF_LOOSE_BALLS_RECOVERED',
                'LOOSE_BALLS_RECOVERED', 'PCT_LOOSE_BALLS_RECOVERED_OFF', 'PCT_LOOSE_BALLS_RECOVERED_DEF',
                'OFF_BOXOUTS', 'BOX_OUTS', 'BOX_OUT_PLAYER_TEAM_REBS', 'BOX_OUT_PLAYER_REBS', 'PCT_BOX_OUTS_OFF',
                'PCT_BOX_OUTS_DEF', 'PCT_BOX_OUTS_TEAM_REB'], inplace=True)

# Remove players not in df
diff = list(set(df['PLAYER_ID'].tolist()) - set(def_data['PLAYER_ID'].tolist()))
df = df[~df['PLAYER_ID'].isin(diff)]
df.sort_values(by='PLAYER_NAME', inplace=True)

# Insert hustle data
def_data = def_data.join(df[df.columns[2:]])

# Set season ID
def_data['SEASON_ID'] = '220' + season[2:4]

def_data.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'AGE', 'GP', 'G', 'D_FGM', 'D_FGA',
       'D_FG_PCT', 'NORMAL_FG_PCT', 'PCT_PLUSMINUS', 'W', 'L', 'MIN', 'STL',
       'BLK', 'DREB', 'CONTESTED_SHOTS', 'CONTESTED_SHOTS_2PT',
       'CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 'CHARGES_DRAWN', 'DEF_BOXOUTS',
       'PCT_BOX_OUTS_REB', 'SEASON_ID'],
      dtype='object')

# Defensive Ratings

In [66]:
r = requests.get('https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Defense&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [67]:
r_dict = json.loads(r.content)

df = pd.DataFrame(data=r_dict['resultSets'][0]['rowSet'], columns=r_dict['resultSets'][0]['headers'])
df.drop(columns=['NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'DREB', 'PCT_DREB', 'STL',
        'BLK', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'DEF_RATING_RANK', 'DREB_RANK', 'DREB_PCT_RANK',
        'PCT_DREB_RANK', 'STL_RANK', 'PCT_STL_RANK', 'BLK_RANK', 'PCT_BLK_RANK', 'OPP_PTS_OFF_TOV_RANK',
        'OPP_PTS_2ND_CHANCE_RANK', 'OPP_PTS_FB_RANK', 'OPP_PTS_PAINT_RANK', 'DEF_WS_RANK', 'CFID', 'CFPARAMS'], inplace=True)
diff = list(set(df['PLAYER_ID'].tolist()) - set(def_data['PLAYER_ID'].tolist()))
df = df[~df['PLAYER_ID'].isin(diff)]

df.sort_values(by='PLAYER_NAME', inplace=True)

# Insert rating data
def_data = def_data.join(df[df.columns[2:]])

def_data.shape

(577, 33)

In [68]:
def_data.to_csv(loc + season + '_def_stats.csv', index=False)